In [ ]:
# Import statements to integrate notebook with azure datastores and datasets
import azureml.core
from azureml.core import Workspace, Datastore
import os
from azureml.core import Dataset
import pandas as pd 

# workspace config information 
ws = Workspace.from_config()

# Details to access the blob storage 
blob_datastoreName='workspaceblobstore' # Name of the datastore to workspace
containerName="azureml-blobstore-d845fdc7-5370-4245-acd8-5ecb6fe12273"# Name of Azure blob container
accountName= "airporttraffic6785036281" # Storage account name
accountKey= "5MIUzuKK7cvU7dHdD3EetRe5kU98EziZYhEqQM15Hv5bD9HjM5IzOtjq7uBpHFJEtdOj35Rrm9dSbnY0A2hjKw==" # Storage account access key

# Using Azure Blob GET API to fetch the blob datastore 
blob_datastore = Datastore.get(ws, blob_datastoreName)

# Get the registered tabular dataset by name 
ds = Dataset.get_by_name(ws, 'covid_impact_on_airport_traffic.csv')
# Converting dataset into pandas dataframe
df = ds.to_pandas_dataframe()

In [ ]:
# Preparing Data 

# Data Attribute Description 
# Percentage Baseline : Proportion of trips on this date as compared to
# Avg number of trips on the same day of week in baseline period 

# Adding Weekday and month 
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()

# Splitting Longitude and Latitudes 
df['long'] = df['Centroid'].apply(lambda x: x[6:-1].split(' ')[0])
df['lat'] = df['Centroid'].apply(lambda x: x[6:-1].split(' ')[1])

# Dropping unnecessary columns 
df.drop(columns=['Path', 'AggregationMethod', 'Version', 'Centroid'], inplace=True)

In [ ]:
print("Dataframe shape ", df.shape)
df.isna().sum() # the number of null values or undefined values

In [ ]:
# Displaying the basic stats 
print("Descriptive Stats")
print(df.describe())
print("-----------------")
print("Print dataframe information")
print(df.info())

In [18]:
# Generating a correlation matrix to understabd 
# The dependencies between the different attributes. 

# Encoding Variables 
def Encode(df):
        for column in df.columns[~df.columns.isin(['PercentOfBaseline','Date'])]:
            df[column] = df[column].factorize()[0]
        return df

In [19]:
encoded_df = Encode(df.copy())
encoded_df.head()

,Date,AirportName,PercentOfBaseline,City,State,ISO_3166_2,Country,Geography,Day,Month,long,lat
0,2020-04-03,0,64,0,0,0,0,0,0,0,0,0
1,2020-04-13,0,29,0,0,0,0,0,1,0,0,0
2,2020-07-10,0,54,0,0,0,0,0,0,1,0,0
3,2020-09-02,0,18,0,0,0,0,0,2,2,0,0
4,2020-10-31,0,22,0,0,0,0,0,3,3,0,0
